<a href="https://colab.research.google.com/github/PENROG21/PySpark/blob/main/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D0%B0_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Выполнение практических заданий

Импорт

In [1]:
!pip install pyspark

# Самостоятельное задание  №1. Погода
У вас есть данные о погоде за несколько лет, собранные из различных метеостанций. Ваша задача — проанализировать эти данные, чтобы выявить тренды и аномалии.

Данные о погоде представлены в формате CSV и содержат следующие столбцы:

`station_id`: ID метеостанции
`date`: Дата наблюдения (в формате YYYY-MM-DD)
`temperature`: Средняя температура в градусах Цельсия

`precipitation`: Количество осадков в миллиметрах

`wind_speed`: Средняя скорость ветра в метрах в секунду

Выполните следующие пункты по порядку.

1. Чтение данных:

Загрузите данные из CSV файла в DataFrame. Скачать CSV нужно по ссылке - https://disk.yandex.ru/d/7Y7ZQgxUKizQsw

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Read file Example").getOrCreate()

df = spark.read.csv("/content/weather_data (2).csv", header=True, inferSchema=True)

# Печать схемы DataFrame
df.printSchema()

# Показ первых 5 строк
df.show(5)

root
 |-- station_id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- temperature: double (nullable = true)
 |-- precipitation: double (nullable = true)
 |-- wind_speed: double (nullable = true)

+----------+----------+------------------+------------------+-----------------+
|station_id|      date|       temperature|     precipitation|       wind_speed|
+----------+----------+------------------+------------------+-----------------+
| station_7|2022-06-28|-6.751842212861652| 23.67004407474563|5.458999894629757|
| station_4|2020-04-07| -9.57484426026233|2.9858244485444665|6.828505392085966|
| station_8|2018-12-22| 19.34342035369938| 33.58211407730149|8.975576079892296|
| station_5|2021-09-09|30.880953114964086|29.110437988411558|18.26465360842913|
| station_7|2023-02-07|23.459048180599673|49.510444679381074|3.787868357696922|
+----------+----------+------------------+------------------+-----------------+
only showing top 5 rows



Найдите топ-5 самых жарких дней за все время наблюдений.

In [ ]:
from pyspark.sql.functions import desc


df_new = df.orderBy(("date")).limit(5)
df_new.show(5)

+----------+----------+-------------------+------------------+------------------+
|station_id|      date|        temperature|     precipitation|        wind_speed|
+----------+----------+-------------------+------------------+------------------+
| station_6|2018-01-02| 29.741063952729462|22.192959421065055| 9.566709633411117|
| station_7|2018-01-02|  7.865994173532567| 33.94488625389202| 17.45310144352111|
| station_9|2018-01-02| 39.432761966208005| 5.721575930617928|  6.54841917466896|
|station_10|2018-01-03|-13.283844378708181| 34.78987388277224|15.618387127721808|
| station_9|2018-01-10| 12.947820176630358| 7.799043184946658| 5.319953861853159|
+----------+----------+-------------------+------------------+------------------+



Найдите метеостанцию с наибольшим количеством осадков за последний год.

In [ ]:
from pyspark.sql.functions import year, col, month

def_yeas = df.filter(year(col("date")) == 2023)

In [ ]:
def_yeas.show()

+----------+----------+-------------------+------------------+------------------+
|station_id|      date|        temperature|     precipitation|        wind_speed|
+----------+----------+-------------------+------------------+------------------+
| station_7|2023-02-07| 23.459048180599673|49.510444679381074| 3.787868357696922|
| station_7|2023-04-13| 19.244547722319886|32.547866755903634|18.095326833602357|
| station_5|2023-01-29|-11.971775823230677|42.359721951679084| 9.247823538250815|
| station_6|2023-04-19| 14.928893786798334| 49.08266619262239| 14.34241136944387|
|station_10|2023-10-24| -16.12299920935483| 36.24905849620262|14.594238129948444|
| station_2|2023-06-14| 28.641155679175974|17.713388242052332|12.644763017357219|
| station_2|2023-02-23|  3.543623291708471|31.045429510885402| 7.102632955076251|
|station_10|2023-09-11|  6.596385246617096| 10.95587397349692|15.670001616905134|
| station_8|2023-08-28| 19.542132052220843|10.697851934411846| 4.461241230855137|
| station_5|2023

In [ ]:
from pyspark.sql.functions import sum, avg

In [ ]:
precipitation_by_station = def_yeas.groupBy("station_id").agg(sum("temperature").alias("ww"))
precipitation_by_station.show(4)

+----------+-----------------+
|station_id|               ww|
+----------+-----------------+
| station_3|175.8907854025927|
| station_2|224.0020087541091|
| station_6|54.90319119179823|
| station_1|44.37624664471266|
+----------+-----------------+
only showing top 4 rows



Подсчитайте среднюю температуру по месяцам за все время наблюдений.

In [ ]:
def_moth = df.groupBy(month(col('date'))).agg(avg('temperature').alias("avg_temperature")).orderBy('month(date)')
def_moth.show()

+-----------+------------------+
|month(date)|   avg_temperature|
+-----------+------------------+
|          1|11.356518462550754|
|          2| 9.067229891101926|
|          3| 7.244080205633994|
|          4|12.024529009744693|
|          5| 9.902883346912718|
|          6|13.421092297254138|
|          7|6.1857183016954576|
|          8|  10.9678002814186|
|          9| 9.596744236573942|
|         10|  9.09884344821895|
|         11| 7.265889994697494|
|         12|11.218592100674337|
+-----------+------------------+



# Самостоятельное задание  №2. Книги и авторы
У вас есть два набора данных: информация о книгах и информация об авторах. Вам нужно объединить эти данные и провести анализ, чтобы найти различные статистики о книгах и авторах.

Данные
Таблица books:

`book_id`: ID книги
`title`: Название книги
`author_id`: ID автора
`genre`: Жанр книги
`price`: Цена книги
`publish_date`: Дата публикации (в формате YYYY-MM-DD)

Таблица authors:

`author_id`: ID автора
`name`: Имя автора
`birth_date`: Дата рождения автора (в формате YYYY-MM-DD)
`country`: Страна автора

Загрузка данных по авторам и книгам

In [ ]:
df_author = spark.read.csv("/content/authors.csv", header=True, inferSchema=True)

In [ ]:
df_book = spark.read.csv("/content/books.csv", header=True, inferSchema=True)

In [ ]:
df_author.printSchema()
df_author.show()

root
 |-- author_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- birth_date: date (nullable = true)
 |-- country: string (nullable = true)

+---------+---------+----------+---------+
|author_id|     name|birth_date|  country|
+---------+---------+----------+---------+
|        1| Author_1|1960-12-31|    India|
|        2| Author_2|1965-12-31|   Canada|
|        3| Author_3|1970-12-31|      USA|
|        4| Author_4|1975-12-31|       UK|
|        5| Author_5|1980-12-31|      USA|
|        6| Author_6|1985-12-31|      USA|
|        7| Author_7|1990-12-31|      USA|
|        8| Author_8|1995-12-31|Australia|
|        9| Author_9|2000-12-31|Australia|
|       10|Author_10|2005-12-31|    India|
+---------+---------+----------+---------+



In [ ]:
df_book.printSchema()
df_book.show()

root
 |-- book_id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- author_id: integer (nullable = true)
 |-- genre: string (nullable = true)
 |-- price: double (nullable = true)
 |-- publish_date: date (nullable = true)

+-------+-------+---------+-----------+-----+------------+
|book_id|  title|author_id|      genre|price|publish_date|
+-------+-------+---------+-----------+-----+------------+
|      1| Book_1|        2|    Mystery|73.57|  1980-12-31|
|      2| Book_2|        1|Non-Fiction| 41.1|  1982-12-31|
|      3| Book_3|       10|    Fiction|10.63|  1984-12-31|
|      4| Book_4|        9|Non-Fiction|46.31|  1986-12-31|
|      5| Book_5|        7|    Science|31.13|  1988-12-31|
|      6| Book_6|        4|Non-Fiction| 83.7|  1990-12-31|
|      7| Book_7|        6|Non-Fiction|40.36|  1992-12-31|
|      8| Book_8|        2|Non-Fiction|84.48|  1994-12-31|
|      9| Book_9|        7|    Fantasy|10.05|  1996-12-31|
|     10|Book_10|        2|    Science| 37.7|  1998

3. Объединение данных:

Объедините таблицы books и authors по author_id.

In [ ]:
df_author_books = df_author.join(df_book, df_book.author_id == df_author.author_id, "inner")

In [ ]:
df_author_books.show()

+---------+---------+----------+---------+-------+-------+---------+-----------+-----+------------+
|author_id|     name|birth_date|  country|book_id|  title|author_id|      genre|price|publish_date|
+---------+---------+----------+---------+-------+-------+---------+-----------+-----+------------+
|        2| Author_2|1965-12-31|   Canada|      1| Book_1|        2|    Mystery|73.57|  1980-12-31|
|        1| Author_1|1960-12-31|    India|      2| Book_2|        1|Non-Fiction| 41.1|  1982-12-31|
|       10|Author_10|2005-12-31|    India|      3| Book_3|       10|    Fiction|10.63|  1984-12-31|
|        9| Author_9|2000-12-31|Australia|      4| Book_4|        9|Non-Fiction|46.31|  1986-12-31|
|        7| Author_7|1990-12-31|      USA|      5| Book_5|        7|    Science|31.13|  1988-12-31|
|        4| Author_4|1975-12-31|       UK|      6| Book_6|        4|Non-Fiction| 83.7|  1990-12-31|
|        6| Author_6|1985-12-31|      USA|      7| Book_7|        6|Non-Fiction|40.36|  1992-12-31|


Найдите топ-5 авторов, книги которых принесли наибольшую выручку.

In [ ]:
df_author_books.groupBy("author_id").agg(sum("price")).alias("sum").show()

+---------+----------+
|author_id|sum(price)|
+---------+----------+
|        1|    111.86|
|        6|     87.93|
|        5|     88.83|
|        9|     46.31|
|        4|      83.7|
|        8|    107.16|
|        7|    132.66|
|       10|     42.33|
|        2|    231.97|
+---------+----------+



In [ ]:
from pyspark.sql.functions import col

df_book = df_book.withColumnRenamed("author_id", "book_author_id")

df_author_books = df_author.join(df_book, df_book.book_author_id == df_author.author_id, "inner")

df_author_books.show() # Выведет DataFrame с переименованным столбцом

+---------+---------+----------+---------+-------+-------+--------------+-----------+-----+------------+
|author_id|     name|birth_date|  country|book_id|  title|book_author_id|      genre|price|publish_date|
+---------+---------+----------+---------+-------+-------+--------------+-----------+-----+------------+
|        2| Author_2|1965-12-31|   Canada|      1| Book_1|             2|    Mystery|73.57|  1980-12-31|
|        1| Author_1|1960-12-31|    India|      2| Book_2|             1|Non-Fiction| 41.1|  1982-12-31|
|       10|Author_10|2005-12-31|    India|      3| Book_3|            10|    Fiction|10.63|  1984-12-31|
|        9| Author_9|2000-12-31|Australia|      4| Book_4|             9|Non-Fiction|46.31|  1986-12-31|
|        7| Author_7|1990-12-31|      USA|      5| Book_5|             7|    Science|31.13|  1988-12-31|
|        4| Author_4|1975-12-31|       UK|      6| Book_6|             4|Non-Fiction| 83.7|  1990-12-31|
|        6| Author_6|1985-12-31|      USA|      7| Book

Найдите количество книг в каждом жанре.

In [ ]:
from pyspark.sql.functions import count, desc

In [ ]:
df_book.groupBy("genre").agg(count("genre").alias("number")).orderBy(desc("number")).show()

+-----------+------+
|      genre|number|
+-----------+------+
|Non-Fiction|     9|
|    Science|     3|
|    Fiction|     3|
|    Fantasy|     3|
|    Mystery|     2|
+-----------+------+



Подсчитайте среднюю цену книг по каждому автору.

In [ ]:
df_author_books.groupBy("name").agg(avg("price").alias("PRICE")).orderBy(desc("PRICE")).show()

+---------+-----------------+
|     name|            PRICE|
+---------+-----------------+
| Author_5|            88.83|
| Author_4|             83.7|
| Author_2|          57.9925|
| Author_9|            46.31|
| Author_7|            44.22|
| Author_6|           43.965|
| Author_1|37.28666666666667|
| Author_8|            35.72|
|Author_10|           21.165|
+---------+-----------------+



Найдите книги, опубликованные после 2000 года, и отсортируйте их по цене.

In [ ]:
df_author_books.filter(year(col("publish_date")) > 2000).orderBy(desc("price")).show()

+---------+--------+----------+---------+-------+-------+--------------+-----------+-----+------------+
|author_id|    name|birth_date|  country|book_id|  title|book_author_id|      genre|price|publish_date|
+---------+--------+----------+---------+-------+-------+--------------+-----------+-----+------------+
|        7|Author_7|1990-12-31|      USA|     20|Book_20|             7|    Mystery|91.48|  2018-12-31|
|        5|Author_5|1980-12-31|      USA|     19|Book_19|             5|    Science|88.83|  2016-12-31|
|        8|Author_8|1995-12-31|Australia|     15|Book_15|             8|    Fantasy| 60.0|  2008-12-31|
|        6|Author_6|1985-12-31|      USA|     17|Book_17|             6|    Fantasy|47.57|  2012-12-31|
|        1|Author_1|1960-12-31|    India|     18|Book_18|             1|Non-Fiction|43.92|  2014-12-31|
|        2|Author_2|1965-12-31|   Canada|     16|Book_16|             2|    Fiction|36.22|  2010-12-31|
|        8|Author_8|1995-12-31|Australia|     12|Book_12|       

# Самостоятельное задание  №3. Фильмы и актеры. С использованием временных таблиц и SQL
У вас есть три набора данных: информация о фильмах и информация о актерах, снявшихся в фильмах, а также id фильма и id актера. Вам нужно объединить эти данные и провести анализ с использованием Spark SQL.

Данные
Таблица movies:

movie_id: ID фильма
title: Название фильма
genre: Жанр фильма
release_date: Дата выхода (в формате YYYY-MM-DD)
budget: Бюджет фильма
Таблица actors:

actor_id: ID актера
name: Имя актера
birth_date: Дата рождения актера (в формате YYYY-MM-DD)
country: Страна актера
Таблица movie_actors:

movie_id: ID фильма
actor_id: ID актера
Выполните следующие пункты по порядку.

1. Чтение данных:

Загрузите данные из CSV файлов в Spark DataFrame. Все что необходимо скачать находится по ссылке - https://disk.yandex.ru/d/EJU5clkFkhWklA

In [2]:
from pyspark.sql import SparkSession


spark = SparkSession.builder.appName("nomber 3").getOrCreate()

df_actors = spark.read.csv("/content/actors.csv", header=True, inferSchema=True)
df_movie_actors = spark.read.csv("/content/movie_actors.csv", header=True, inferSchema=True)
df_movies = spark.read.csv("/content/movies.csv", header=True, inferSchema=True)

2. Создание временных таблиц:

Создайте временные таблицы для данных о фильмах, актерах и связях между ними.

In [6]:
df_actors.createOrReplaceTempView("actors")
df_movie_actors.createOrReplaceTempView("movie_actors")
df_movies.createOrReplaceTempView("movies")

In [22]:
df_actors.show()

+--------+--------+----------+---------+
|actor_id|    name|birth_date|  country|
+--------+--------+----------+---------+
|       1| Actor_1|1960-12-31|   Canada|
|       2| Actor_2|1962-12-31|       UK|
|       3| Actor_3|1964-12-31|       UK|
|       4| Actor_4|1966-12-31|       UK|
|       5| Actor_5|1968-12-31|    India|
|       6| Actor_6|1970-12-31|      USA|
|       7| Actor_7|1972-12-31|    India|
|       8| Actor_8|1974-12-31|Australia|
|       9| Actor_9|1976-12-31|      USA|
|      10|Actor_10|1978-12-31|Australia|
|      11|Actor_11|1980-12-31|      USA|
|      12|Actor_12|1982-12-31|    India|
|      13|Actor_13|1984-12-31|       UK|
|      14|Actor_14|1986-12-31|   Canada|
|      15|Actor_15|1988-12-31|       UK|
|      16|Actor_16|1990-12-31|    India|
|      17|Actor_17|1992-12-31|      USA|
|      18|Actor_18|1994-12-31|       UK|
|      19|Actor_19|1996-12-31|    India|
|      20|Actor_20|1998-12-31|Australia|
+--------+--------+----------+---------+
only showing top

In [7]:
df_movie_actors.show()

+--------+--------+
|movie_id|actor_id|
+--------+--------+
|       1|      25|
|      16|       5|
|       6|      16|
|      16|      11|
|      14|      21|
|       3|       6|
|      15|       9|
|       3|      13|
|       2|      24|
|       1|       8|
|       9|      14|
|       9|      24|
|       7|       1|
|       3|      17|
|      18|      24|
|      11|       5|
|       7|      25|
|       9|       2|
|       1|      25|
|      14|      28|
+--------+--------+
only showing top 20 rows



In [9]:
df_movies.show()

+--------+--------+------+------------+-------------+
|movie_id|   title| genre|release_date|       budget|
+--------+--------+------+------------+-------------+
|       1| Movie_1|Horror|  2000-12-31|8.660058311E7|
|       2| Movie_2|Comedy|  2001-12-31|1.274740083E7|
|       3| Movie_3|Action|  2002-12-31| 1.80157747E7|
|       4| Movie_4| Drama|  2003-12-31|4.817612061E7|
|       5| Movie_5| Drama|  2004-12-31| 7.40501611E7|
|       6| Movie_6|Action|  2005-12-31|1.476121831E7|
|       7| Movie_7| Drama|  2006-12-31|4.456703643E7|
|       8| Movie_8| Drama|  2007-12-31|4.880227617E7|
|       9| Movie_9|Action|  2008-12-31|2.201627853E7|
|      10|Movie_10|Action|  2009-12-31|1.244027929E7|
|      11|Movie_11|Comedy|  2010-12-31|8.380567138E7|
|      12|Movie_12|Comedy|  2011-12-31|5.074409933E7|
|      13|Movie_13|Action|  2012-12-31|   2423742.36|
|      14|Movie_14|Sci-Fi|  2013-12-31|8.049514883E7|
|      15|Movie_15| Drama|  2014-12-31|9.809858674E7|
|      16|Movie_16|Comedy|  

In [12]:
df = spark.sql("""
SELECT movies.genre, COUNT(*) AS number
FROM movies
GROUP BY movies.genre
ORDER BY number DESC
LIMIT 5;
""")

In [13]:
df.show()

+------+------+
| genre|number|
+------+------+
| Drama|     6|
|Action|     6|
|Comedy|     4|
|Horror|     2|
|Sci-Fi|     2|
+------+------+



Найдите актера с наибольшим количеством фильмов.

In [16]:
df_2 = spark.sql("""
SELECT a.name, COUNT(ma.movie_id) AS film_count
FROM actors a
JOIN movie_actors ma ON a.actor_id = ma.actor_id
GROUP BY a.actor_id, a.name
ORDER BY film_count DESC
LIMIT 1;
""")

In [17]:
df_2.show()

+--------+----------+
|    name|film_count|
+--------+----------+
|Actor_17|         5|
+--------+----------+



Подсчитайте средний бюджет фильмов по жанрам.

In [20]:
df_3 = spark.sql("""
SELECT movies.genre, AVG(movies.budget)
FROM movies
GROUP BY movies.genre
""")

In [21]:
df_3.show()

+------+--------------------+
| genre|         avg(budget)|
+------+--------------------+
| Drama| 6.076021856166667E7|
|Horror|      8.7281876775E7|
|Comedy|     5.20709662225E7|
|Action|2.7492742561666667E7|
|Sci-Fi|       7.809715175E7|
+------+--------------------+



Найдите фильмы, в которых снялось более одного актера из одной страны.

In [24]:
df_4 = spark.sql("""
SELECT m.title, a.country, COUNT(*) AS actor_count
FROM movies m
JOIN movie_actors ma ON m.movie_id = ma.movie_id
JOIN actors a ON ma.actor_id = a.actor_id
GROUP BY m.title, a.country
HAVING COUNT(*) > 1;
""")

df_4.show()

+--------+---------+-----------+
|   title|  country|actor_count|
+--------+---------+-----------+
| Movie_7|    India|          2|
| Movie_3|      USA|          2|
|Movie_10|       UK|          2|
|Movie_15|    India|          2|
|Movie_18|Australia|          2|
| Movie_1|    India|          3|
| Movie_2|      USA|          2|
| Movie_7|      USA|          2|
|Movie_10|      USA|          2|
+--------+---------+-----------+

